# Notebook M06 - Plume problem

This is an implementation of a 2D viscous flow code. The energy equation (for temperature) is solved at the cell centers (unlike the example in the chapter 10 notebook, which is solved at the basic nodes.

This notebook puts all of the major function definitions into separate source files and contains only the high-level program code.

In [2]:
# Import necessary packages
using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics 
using Random
using NearestNeighbors

include("Grid.jl")
include("GridOperations.jl")
include("Markers.jl")
include("Stokes.jl")
include("StokesCylindrical.jl")

include("Temperature.jl")
include("TemperatureCylindrical.jl")
include("melting/yasuda.jl")

# note that we import pyplot last to avoid a name conflict with grid.
using PyPlot
include("Visualization.jl")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


plots (generic function with 1 method)

# Problem setup

- Lithosphere with initial geotherm set by plate cooling

In [2]:
# problem set-up
using SpecialFunctions
# function to define plate cooling solution
function plate_cooling(Tsurf,Tbtm,yL0,kappa,y,t)
   # y is a vector of y coordinates where we need the solution
    T = Tsurf .+ (Tbtm-Tsurf) .* (y/yL0)
    for n in 1:20 # note 20 is an index in a summation from 1 to Inf
        T += (Tbtm-Tsurf)*2/pi .* 1/n*exp(-kappa*n^2*pi^2*t/yL0^2)*sin(n*pi*y/yL0)
    end
    return T
end

function halfspace_cooling(Tsurf,Tmantle,kappa,y,t)
    if t == 0.0
        if y==0.0
            return Tsurf
        else
            return Tmantle
        end
    else
        T = Tsurf + (Tmantle-Tsurf)*erf(y/2/sqrt(kappa*t))         
        return T
    end
end

function halfspace_cooling_from_thickness(Tsurf,Tmantle,kappa,y,thickness)
    t = (thickness/2)^2/kappa
    return halfspace_cooling(Tsurf,Tmantle,kappa,y,t)
end

# function to compute solidus and liquidus

# function to compute melt fraction

# function to compute viscosity
function viscosity(eta0::Float64,depth::Float64,T::Float64,E::Float64 ; visc_max=1.0e25)
   # E should be given in J/mol/K
   # Expect all temperatures in kelvin.
   Tref = 1350.0+273.0
   R = 8.314 #J/mol/K
   depth_factor = depth > 6.6e5 ? 30.0 : 1.0
   viscosity = eta0*depth_factor*exp(E/R/Tref*(Tref/T-1))
   if viscosity > visc_max
      viscosity = visc_max
   end
   return viscosity
end

struct Materials
    # 1 - ambient mantle
    # 2 - eclogite
    alpha::Vector{Float64}
    rho0::Vector{Float64}
    Hr::Vector{Float64}
    Cp::Vector{Float64}
    kThermal::Vector{Float64}
    eta::Vector{Float64}
    Ea::Vector{Float64}
    function Materials()
         new([3e-5,3e-5],[3300.,3300.],[0.0,0.0],[1000.,1000.],[3.0,3.0],[1e21,1e21],[3e5,3e5])
    end
end

function update_marker_properties!(markers::Markers,materials::Materials)
    rho = markers.scalarFields["rho"]
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    eta = markers.scalarFields["eta"]
    Threads.@threads for i in 1:markers.nmark                
        # re-compute density using the current temperature value
        # assume reference temperature is 273.0
        # markers.scalars[rho,i] = materials.rho0[mmat[i]] # don't update density - for comparison with gerya
        markers.scalars[rho,i] = materials.rho0[mmat[i]]*(1.0-materials.alpha[mmat[i]]*(markers.scalars[T,i]-273.0)) 
        markers.scalars[eta,i] = viscosity(materials.eta[mmat[i]],markers.x[2,i],markers.scalars[T,i],materials.Ea[mmat[i]])
    end
end

function initial_conditions!(markers::Markers,materials::Materials)
    # Define geometric properties
    lithosphere_thickness = 1.5e5
    mantle_temperature = 1350.0
    
    material = markers.integerFields["material"]
    T = markers.scalarFields["T"]
    eta = markers.scalarFields["eta"]
    alpha = markers.scalarFields["alpha"]
    cp = markers.scalarFields["Cp"]
    Hr = markers.scalarFields["Hr"]
    Threads.@threads for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        mr = ((mx-0)^2 + (my-2.85e6)^2)^0.5 
        
        # define initial cmb hot layer geometry
        h = 150e3 + (113e3-150e3)*(mx-0.0)/5e5
                
        # set material - eclogite at cmb
        if my > 2.85e6-h
           markers.integers[material,i] = 2
        else
           markers.integers[material,i] = 1
        end
                
        if my < lithosphere_thickness
            markers.scalars[T,i] = plate_cooling(273.0,1350.0+273.0,1.5e5,1e-6,my,100e6*3.15e7)
        else
            markers.scalars[T,i] = halfspace_cooling_from_thickness(2500.0+273.0,1350.0+273.0,1e-6,2.85e6-my,h)
        end
        ind = markers.integers[material,i]
        markers.scalars[eta,i] = viscosity(materials.eta[ind],markers.x[2,i],markers.scalars[T,i],materials.Ea[ind])
        markers.scalars[alpha,i] = materials.alpha[ind]            
        markers.scalars[cp,i] = materials.Cp[ind]  
        markers.scalars[Hr,i] = materials.Hr[ind]  
    end
    update_marker_properties!(markers,materials)
end

initial_conditions! (generic function with 1 method)

In [3]:
# plotting routine
function plume_plots(grid::CartesianGrid,vx,vy,rho,T)
    # make some diagnostic plots using pyplot
    fig, (ax1,ax2,ax3,ax4) = subplots(1,4)
    ax1.pcolor(grid.xc,grid.yc,rho)
    ax1.invert_yaxis()
    ax1.set_title("Density")
    ax1.set_aspect("equal")
    ax2.pcolor(grid.xc,grid.yc,T)
    ax2.invert_yaxis()
    ax2.set_title("Temperature")
        ax2.set_aspect("equal")

    ax3.pcolor(grid.xc,grid.yc,vx)
    ax3.invert_yaxis()
    ax3.set_title("vx")
        ax3.set_aspect("equal")

    ax4.pcolor(grid.xc,grid.yc,vy)
    ax4.invert_yaxis()
    ax4.set_title("vy")
        ax4.set_aspect("equal")

    fig.show()
end


plume_plots (generic function with 1 method)

# i2vis procedure

0. Set up the Eulerian grid and Lagrangian markers. Distribute the markers within the domain and assign initial conditions.
1. Compute density and viscosity at the basic nodes. Compute cell center viscosity via averaging.
2. Assemble and solve the Stokes equations.
3. Compute the maximum timestep from the velocity field and CFL number.
3. Transfer temperature from markers to basic nodes.
4. Compute the new temperature solution.
5. If the maximum temperature change exceeds a specified value, reduce the timstep and repeat previous step.
6. Compute the subgrid temperature changes and update the temperature on the markers.
7. Move the markers using 4th order Runge-Kutta

In [4]:
function plume_model(;max_step::Int64=-1,max_time::Float64=-1.0)
    nx = 101#101
    ny = 141#285
    W = 1e6
    H = 2.850e6
    gx = 0.0
    gy = 10.0

    Tbcval = [0.0,0.0,273.0,2500.0+273.0]
    bc = BoundaryConditions(0,0,0,0) # currently does nothing but is required argument to stokes solver.
    materials = Materials()
    
    markx = 6
    marky = 12
    target_markers = markx*marky
    min_markers = Int(floor(target_markers*0.75))
    max_markers = Int(ceil(target_markers*1.5))

    seconds_in_year = 3.15e7
    plot_interval = 5e5*seconds_in_year # plot interval in seconds
    end_time::Float64 = max_time == -1.0 ? typemax(Float64) : max_time
    max_step::Int64 = max_step == -1 ? typemax(Int64) : max_step
    
    dtmax = plot_interval
    grid = CartesianGrid(W,H,nx,ny)
    println("Creating Markers...")
    @time markers = Markers(grid,["alpha","Cp","T","rho","eta","Hr"],["material"] ; nmx=markx,nmy=marky,random=true)
    println("Initial condition...")
    @time initial_conditions!(markers, materials)

    # define arrays for k, rho, cp, H at the basic nodes. Fill them with constant values for now.
    kThermal = 3.0 .*ones(grid.ny,grid.nx);

    time = 0.0
    iout=0
    last_plot = 0.0
    rho_c = nothing
    dt = 1e10

    local rho_c
    local rho_vx
    local rho_vy
    local alpha
    #       local Hr
    local Cp_c
    local eta_s
    local eta_n
    local eta_vx
    local eta_vy
    local vx,vy
    local vxc=nothing
    local vyc=nothing
    local T
    local dt
    local dTmax
    local dTemp
    local Tnew=nothing
    local Tlast

    output_dir = "plume_1"
    itime=1

    local terminate = false
    while !terminate
        update_marker_properties!(markers,materials)
        # 1. Transfer properties markers -> nodes
        visc_method = "logarithmic"

        eta_s_new, = marker_to_stag(markers,grid,["eta",],"basic",method=visc_method);
        # 1b. Cell Centers
        rho_c_new,Cp_c_new,alpha_new,Tlast_new = marker_to_stag(markers,grid,["rho","Cp","alpha","T"],"center")
        eta_n_new, = marker_to_stag(markers,grid,["eta",],"center",method=visc_method);
        # 1c. Vx and Vy nodes:        
        eta_vx_new, = marker_to_stag(markers,grid,["eta"],"vx",method=visc_method)
        eta_vy_new, = marker_to_stag(markers,grid,["eta"],"vy",method=visc_method)
        rho_vx_new, = marker_to_stag(markers,grid,["rho",],"vx")
        rho_vy_new, = marker_to_stag(markers,grid,["rho",],"vy")
        
        # deal with any NaN values from interpolation:
        if itime > 1
            if any(isnan.(eta_s_new))
                println("found nan values")
            end
            replace_nan!(eta_s,eta_s_new)
            replace_nan!(rho_c,rho_c_new)
#             replace_nan!(Hr,Hr_new)
            replace_nan!(Cp_c,Cp_c_new)
            replace_nan!(alpha,alpha_new)
            replace_nan!(eta_n,eta_n_new)
            replace_nan!(Tlast,Tlast_new)
            replace_nan!(rho_vx,rho_vx_new)
            replace_nan!(rho_vy,rho_vy_new)
            replace_nan!(eta_vx,eta_vx_new)
            replace_nan!(eta_vy,eta_vy_new)
        end
        # Copy field data 
        rho_vx = copy(rho_vx_new)
        rho_vy = copy(rho_vy_new)
        eta_vx = copy(eta_vx_new)
        eta_vy = copy(eta_vy_new)
        rho_c = copy(rho_c_new)
#         Hr = copy(Hr_new)
        Cp_c = copy(Cp_c_new)
        alpha = copy(alpha_new)
        eta_s = copy(eta_s_new)
        eta_n = copy(eta_n_new)
        Tlast = copy(Tlast_new)

        Tlast = ghost_temperature_center(grid,Tlast,Tbcval)
        # 2. Assemble and solve the stokes equations
        #L,R = form_stokes(grid,eta_s,eta_n,rho_vx,rho_vy,bc,gx,gy,dt=dt)
        L,R = form_stokes_cylindrical(grid,eta_s,eta_n,eta_vx,eta_vy,rho_vx,rho_vy,bc,gx,gy)
        stokes_solution = L\R
        vx,vy,P = unpack(stokes_solution,grid;ghost=true)
    
        # Get the velocity at the cell centers:
        vxc,vyc = velocity_to_centers(grid,vx,vy)
        adiabatic_heating = compute_adiabatic_heating(grid,rho_c,Tlast,alpha,gx,gy,vxc,vyc)
        shear_heating = compute_shear_heating(grid,vx,vy,eta_n,eta_s)
         H = (adiabatic_heating .+ shear_heating).*0.0
    
        # 3. Compute the advection timestep:
        if itime > 1
            this_dtmax = min(1.2*dt,dtmax)
        else
            this_dtmax = dtmax
        end
        dt = compute_timestep(grid,vxc,vyc ; dtmax=this_dtmax)
        
        dTmax = Inf
        dTemp = nothing
        Tnew = nothing
        titer=1
        for titer=1:2# limit maximum temperature change
            # assemble and solve the energy equation
            println("Trying with timestep ",dt/3.15e7/1e6," Myr")
            L,R = assemble_energy_equation_cylindrical(grid,rho_c,Cp_c,kThermal,H,Tlast,dt,Tbcval);
            Tnew = L\R;
            Tnew = reshape(Tnew,grid.ny,grid.nx);
            Tnew = ghost_temperature_center(grid,Tnew,Tbcval);

            T = copy(Tnew)

            dTemp = Tnew-Tlast
            # compute the maximum temperature change
            dTmax = maximum(abs.(dTemp[2:end-1,2:end-1]))
            println("dTmax=",dTmax," dt=",dt/3.15e7/1e6)
            dt = min(dt,dTmax < 10.0 ? dt : dt*10.0/dTmax)
            if dTmax < 10.0
                break
            end
        end
        if any(isnan.(markers.scalars[markers.scalarFields["rho"],:]))
            println("nan in density")
            break
        end
        dT_subgrid_node = subgrid_temperature_relaxation_center!(markers,grid,dTemp,Cp_c[1,1],kThermal[1,1],dt)
        dT_remaining = dTemp - dT_subgrid_node

        cell_center_change_to_markers!(markers,grid,dT_remaining,"T")
#         println("dT_remaining max ",extrema(dT_remaining))
#         println("marker temperature max ",maximum(markers.scalars[markers.scalarFields["T"],:])," min ",minimum(markers.scalars[markers.scalarFields["T"],:]))
#         println("marker density max ",maximum(markers.scalars[markers.scalarFields["rho"],:])," min ",minimum(markers.scalars[markers.scalarFields["rho"],:]))

        if any(isinf.(markers.scalars[markers.scalarFields["T"],:]))
            println("Nan in marker temp")
            break
        end
        if time >= max_time || itime >= max_step
            terminate = true
        end
                
        if time == 0.0 || time - last_plot >= plot_interval || terminate
            last_plot = time 
    
            # plume_plots(grid,vxc,vyc,rho_c,Tnew)
    
            name = @sprintf("%s/viz.%04d.vtr",output_dir,iout)
            println("Writing visualization fle ",name)
            vn = velocity_to_basic_nodes(grid,vxc,vyc)
            @time visualization(grid,rho_c,eta_s,vn,P,Tnew[2:end-1,2:end-1],time/seconds_in_year;filename=name)
            
            name = @sprintf("%s/markers.%04d.vtp",output_dir,iout)
            println("Writing visualization fle ",name)

            #@time Threads.@spawn 
            visualization(markers,time/seconds_in_year;filename=name)
            iout += 1
        end
        add_remove_markers!(markers,grid,Tnew,min_markers,target_markers,max_markers)
        println("Min/Max velocity: ",minimum(vyc)," ",maximum(vyc))
        move_markers_rk4!(markers,grid,vx,vy,dt,continuity_weight=1.0/3.0)
        time += dt
        itime += 1
        println("Finished Step ",itime," time=",time/seconds_in_year/1e6," Myr")
     end
     return grid,markers,vx,vy,vxc,vyc,rho_c,dTemp,Tnew,Tlast,time
 end



plume_model (generic function with 1 method)

In [5]:
# run the model for a few timesteps.
@time grid,markers,vx,vy,vxc,vyc,rho_c,dTemp,Tnew,Tlast,time = plume_model(max_time=100e6*3.15e7);
# divv = compute_velocity_divergence(grid,vx,vy)
# figure()
# pcolor(divv)
# colorbar()
# show()

Creating Markers...
  0.100762 seconds (19 allocations: 94.594 MiB, 72.99% gc time)
Initial condition...
  0.905596 seconds (728.52 k allocations: 39.263 MiB, 93.32% compilation time: 13% of which was recompilation)
Trying with timestep 0.29872048929046396 Myr
dTmax=3.3090316733873806 dt=0.29872048929046396
Writing visualization fle plume_1/viz.0000.vtr
  1.400532 seconds (579.26 k allocations: 28.894 MiB, 96.27% compilation time)
Writing visualization fle plume_1/markers.0000.vtp
  0.148752 seconds (9.00 k allocations: 15.845 MiB, 86.45% gc time, 6.05% compilation time)
keeping 1008000 markers nmark=1008000
Min/Max velocity: -1.7073703079163248e-10 6.993633612062228e-11
Finished Step 2 time=0.29872048929046396 Myr
Trying with timestep 0.28908337328615896 Myr
dTmax=3.5519344918711795 dt=0.28908337328615896
keeping 1008000 markers nmark=1008000
Min/Max velocity: -1.9228107656686502e-10 8.326776285773592e-11
Finished Step 3 time=0.587803862576623 Myr
Trying with timestep 0.28026604375900

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
figure()
scatter(markers.x[1,1:markers.nmark],markers.x[2,1:markers.nmark],c=isnan.(markers.scalars[markers.scalarFields["rho"],1:markers.nmark]),s=0.1,cmap="jet")
colorbar()
gca().invert_yaxis()
show()

ind = findall(isnan,(markers.scalars[markers.scalarFields["rho"],1:markers.nmark]))
print(ind)
markers.scalars[:,ind]

In [ ]:
# figure()
# # title(print(time/3.15e7))
# # pcolor(grid.xc,grid.yc,eta_n)
# gca().invert_yaxis()
# colorbar()
# show()

figure()
scatter(markers.x[1,1:markers.nmark],markers.x[2,1:markers.nmark],c=(markers.integers[markers.integerFields["material"],1:markers.nmark]),s=0.1)
gca().invert_yaxis()
gca().set_aspect("equal")
colorbar()
show()

figure()
scatter(markers.x[1,1:markers.nmark],markers.x[2,1:markers.nmark],c=(markers.scalars[markers.scalarFields["rho"],1:markers.nmark]),s=0.1)
gca().invert_yaxis()
gca().set_aspect("equal")
colorbar()
show()


figure()
scatter(markers.x[1,1:markers.nmark],markers.x[2,1:markers.nmark],c=(markers.scalars[markers.scalarFields["T"],1:markers.nmark]),s=0.1)
gca().invert_yaxis()
gca().set_aspect("equal")
colorbar()
show()

In [ ]:
# Optionally, run a few steps with the profiler
using ProfileSVG
@profview plume_model(2)

In [ ]:
minimum(markers.scalars[markers.scalarFields["T"],:])

In [ ]:
include("Markers.jl")
test = markers_per_cell(grid,markers)
figure()
pcolor(test)
colorbar()
show()

In [ ]:

a=ones(Bool,10)
sum(a)
a[2] = false
sum(a)

In [ ]:
[10 .*ones(Int,1,10); ones(Int,1,10)] 

In [ ]:
rand(1,10)

